In [1]:
import pandas as pd
import numpy as np
import re

In [306]:
def get_data(day:int)->list:

    data = []
    f = open(f'data/data{day}.txt', 'r')
    for line in f:
        data.append(line)
    data = [i.replace('\n','') for i in data]
    return data

#day1
def compare_if_increased(data:list)->list:
    '''
    if the value in a list is higher than the prev value then return inc, else return idk
    '''
    res = []
    for i in range(len(data)-1):
        if data[i] < data[i+1]:
            res.append('inc')
        else:
            res.append('idk')
            
    return res

#day2
def get_int_from_string(string):
    i = ''.join(x for x in string if x.isdigit())
    return int(i)


#day3
def get_rating(data, method):
    keep = data.copy()
    n = len(keep)
    for i in range(12):
        k_ = keep.mode()[i].fillna('-1').max()
        if method =='mode':
            k = k_
        else:
            k = str(abs(int(k_)-1))
        keep = keep.loc[keep[i] == k, :]
        n = len(keep)
        if n ==1: break
    return ''.join(keep.values[0])


#day4
def get_bingo_nums(data):
    nums = data.pop(0).split(',')
    while True: 
        try:
            data.remove('')
        except: break
    return nums, data

def get_bingo_boards(board_data):
    boards = {}
    for i in range(100):
        l = []
        for _ in range(5):
            r = board_data.pop(0).split(' ')
            while True: 
                try:
                    r.remove('')
                except: break
            l.append(r)
        boards[str(i)] = {'board' : pd.DataFrame(l)}
    return boards

def make_bingo(data):
    
    nums, board_data = get_bingo_nums(data)
    boards = get_bingo_boards(board_data)
    
    return nums, boards

def run_bingo(nums, boards):
    winner = []
    numbers_called = []
    for n in nums:
        numbers_called.append(n)
        for i in boards.keys():
            # check if any rows have all values:
            for row in boards[i]['board'].values:
                if len(set(row).intersection(numbers_called)) == 5:
                    winner = boards[i]['board']
                    print(f'BOARD {i} ROW {row } WINS AFTER {len(numbers_called)} NUMBERS')
                    return winner, numbers_called
                    break
            #check if any columns have all values
            for col in boards[i]['board']:
                if len(set(boards[i]['board'][col]).intersection(numbers_called)) == 5:
                    winner = boards[i]['board']
                    print(f'BOARD {i} COLUMN {col} WINS AFTER {len(numbers_called)} NUMBERS')
                    return winner, numbers_called
                    break

def get_bingo_score(winner, numbers_called):
    winner_flat = [int(i) for i in winner.values.flatten()]
    marked = set(winner_flat).intersection(set([int(i) for i in numbers_called]))
    return (sum(winner_flat)-sum(marked)) * int(numbers_called[-1])
    
def run_bingo_loser(nums, boards):
    winner = []
    winners=[]
    numbers_called = []
    for n in nums:
        numbers_called.append(n)
        for i in boards.keys():
            # check if any rows have all values:
            for row in boards[i]['board'].values:
                if len(set(row).intersection(numbers_called)) == 5:
                    winner = boards[i]['board']
                    winners.append(i)
            #check if any columns have all values
            for col in boards[i]['board']:
                if len(set(boards[i]['board'][col]).intersection(numbers_called)) == 5:
                    winner = boards[i]['board']
                    winners.append(i)
            if len(set(winners)) == 100:
                print(f'BOARD {i} ROW {row } LOSES AFTER {len(numbers_called)} NUMBERS')
                return winner, numbers_called, winners
                    

# DAY 1

In [295]:
#p1
data = [int(i) for i in get_data(day=1)]
res = compare_if_increased(data)
res.count('inc')

1226

In [296]:
#p2
data3 = pd.Series(data).rolling(3).sum().dropna().values
res = compare_if_increased(data3)
res.count('inc')

1252

# DAY 2

In [24]:
#p1
data=get_data(day=2)

fw = [get_int_from_string(i) for i in data if 'forward' in i]
up = [get_int_from_string(i) for i in data if 'up' in i]
dn = [get_int_from_string(i) for i in data if 'down' in i]

(sum(dn) - sum(up)) * sum(fw)

1804520

In [25]:
#p2
h = 0
aim = 0
d = 0

for i in data:
    if 'forward' in i:
        i = get_int_from_string(i)
        h += i
        d += i*aim
    elif 'down' in i:   

        aim += get_int_from_string(i)
    elif 'up' in i:    
        aim -= get_int_from_string(i)
h * d

1971095320

# DAY 3

In [73]:
#p1
data = get_data(day=3)
df = pd.DataFrame([[i for i in char] for char in data])

mode = df.mode().values[0]
gr_bit = ''.join(mode)
er_bit = ''.join(['1' if i =='0' else '0' for i in mode])

int(er_bit,2) * int(gr_bit,2)


1307354

In [188]:
#p2
o2 = get_rating(df, method='mode')
co2 = get_rating(df, method='notmode')

int(o2,2) * int(co2,2)

482500

# DAY 4

In [299]:
#p1
data = get_data(day=4)
nums, boards = make_bingo(data)
winner, numbers_called = run_bingo(nums, boards)
get_bingo_score(winner, numbers_called)

BOARD 11 ROW ['49' '69' '76' '98' '22'] WINS AFTER 22 NUMBERS


23177

In [307]:
#p2
loser, numbers_called, winners = run_bingo_loser(nums, boards)
get_bingo_score(loser, numbers_called)

BOARD 59 ROW ['67' '70' '14' '77' '86'] LOSES AFTER 87 NUMBERS


6804

# DAY 5

In [329]:
data = get_data(day=5)

cols = ['x1','y1','x2','y2']
df = pd.DataFrame(data=[re.split(' -> |,',i) for i in data], columns=cols)

df

,x1,y1,x2,y2
0,405,945,780,945
1,253,100,954,801
2,518,300,870,300
3,775,848,20,848
4,586,671,469,671
...,...,...,...,...
495,872,988,371,988
496,102,203,102,123
497,518,338,518,360
498,819,263,328,754


[['405,945', '780,945'],
 ['253,100', '954,801'],
 ['518,300', '870,300'],
 ['775,848', '20,848'],
 ['586,671', '469,671'],
 ['598,20', '900,20'],
 ['51,975', '438,588'],
 ['561,456', '67,456'],
 ['353,135', '882,664'],
 ['357,873', '407,873'],
 ['480,247', '774,247'],
 ['230,895', '230,709'],
 ['590,738', '644,792'],
 ['696,821', '696,198'],
 ['501,461', '85,461'],
 ['884,88', '884,922'],
 ['140,780', '146,780'],
 ['795,208', '273,208'],
 ['256,185', '256,525'],
 ['282,196', '282,85'],
 ['701,133', '18,133'],
 ['623,548', '623,573'],
 ['734,895', '29,190'],
 ['212,944', '934,944'],
 ['948,504', '948,502'],
 ['551,613', '551,27'],
 ['62,54', '452,54'],
 ['915,851', '915,290'],
 ['924,843', '924,145'],
 ['662,412', '422,172'],
 ['465,87', '247,87'],
 ['391,91', '326,91'],
 ['267,572', '267,306'],
 ['84,505', '594,505'],
 ['453,383', '80,10'],
 ['737,891', '784,891'],
 ['599,339', '40,898'],
 ['868,962', '31,125'],
 ['153,661', '705,661'],
 ['760,74', '339,495'],
 ['972,72', '972,714'],
